<a href="https://colab.research.google.com/github/Chevron9/iannwtf/blob/master/8/homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext tensorboard
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import datetime
%matplotlib notebook

In [ ]:
# Load dataset, split into test and training data
train_ds, _ = tfds.load('fashion_mnist', split= ['train', 'test'], data_dir="data\\", as_supervised=True, shuffle_files=True)

In [ ]:
def overview_data():
  train_ds_numpy = tfds.as_numpy(train_ds)

  #Look at shapes images and labels of dataset
  print("length train: ",len(train_ds))

  x = 0
  for i in train_ds_numpy:
    x += 1
    # print(i)
    print("shape ",format(i[0].shape))
    print("max ",np.amax(i[0]))
    print("min ",np.amin(i[0]))
    print(format(i[1]))
    if x == 3:
      break

overview_data()

In [ ]:
train_ds = train_ds.map(lambda image, label: tf.cast((image/255)*2 - 1, tf.float32))

prefetch_size = tf.data.experimental.AUTOTUNE

# batching, shuffle und prefetching, batchsize is 32!
train_ds = train_ds.batch(32).shuffle(buffer_size=32).prefetch(prefetch_size)

In [ ]:
#takes random noise input and returns picture to fool discriminator

class Generator(tf.keras.layers.Layer):
  def __init__(self):
    super(Generator, self).__init__()


    self.dense = tf.keras.layers.Dense(units=7*7*64,
                            activation=None 
                            )
    self.batchnorm_1 = tf.keras.layers.BatchNormalization()
      

      
    self.conv2DTranspose_1 = tf.keras.layers.Conv2DTranspose(filters = 32,
                                         kernel_size = 5,
                                         strides = 2,
                                         padding = 'same',
                                         kernel_initializer = tf.keras.initializers.glorot_normal,
                                         activation=None
                                      )
    self.batchnorm_2 = tf.keras.layers.BatchNormalization()                                         
      

    self.conv2DTranspose_2 = tf.keras.layers.Conv2DTranspose(filters = 16,
                                         kernel_size = 5,
                                         strides = 2,
                                         activation = None,
                                         padding = 'same',
                                         kernel_initializer = tf.keras.initializers.glorot_normal
                                      )
    self.batchnorm_3 = tf.keras.layers.BatchNormalization()                                
      

    self.conv2DTranspose_3 = tf.keras.layers.Conv2D(filters = 1,
                                         kernel_size = 5,
                                         strides = 1,
                                         padding = 'same',
                                         kernel_initializer = tf.keras.initializers.glorot_normal,
                                         activation = tf.nn.tanh
                             )                                  
    

  def call(self, x, training = True): 
    x = self.dense(x)
    x = self.batchnorm_1(x,training)
    x = tf.keras.activations.relu(x)

    x = tf.reshape(x, shape=(-1, 7, 7, 64))

    x = self.conv2DTranspose_1(x)
    x = self.batchnorm_2(x, training)
    x = tf.keras.activations.relu(x)

    x = self.conv2DTranspose_2(x)
    x = self.batchnorm_3(x, training)
    x = tf.keras.activations.relu(x)

    x =  self.conv2DTranspose_3(x)
    return x

In [ ]:
#simple discriminator model, downsamples the input and gives back the probability of input being real
class Discriminator(tf.keras.layers.Layer):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.conv2D_1 = tf.keras.layers.Conv2D(filters = 8,
                             kernel_size = 5,
                             strides = 2,
                             padding = 'same',
                             kernel_initializer = tf.keras.initializers.glorot_normal,
                             activation=None,
                             )


    self.conv2D_2 = tf.keras.layers.Conv2D(filters = 16,
                           kernel_size = 5,
                           strides = 2,
                           padding = 'same',
                           kernel_initializer = tf.keras.initializers.glorot_normal,
                           activation=None,
                           ) 

    self.flatten = tf.keras.layers.Flatten()
       
    self.dense = tf.keras.layers.Dense(units=1, 
                          activation=tf.keras.activations.sigmoid
                          ) #softmax instead? #classification as real is output 1
    
    
  def call(self, x, training = True):  
    x = self.conv2D_1(x)
    x = tf.keras.activations.sigmoid(x) #sigmoid instaed of relu?
    
    x = self.conv2D_2(x)
    x = tf.keras.activations.sigmoid(x) #sigmoid instead of relu?

    x = self.flatten(x)

    x = self.dense(x)
    return x

In [ ]:
#LOSS FUNCTIONS 

def generator_loss(predictions):
    # Get only the output probabilities for the fake images.
    predictions_fake = predictions[:32]
    # Create the label vector indicating that the images are correct (=1).
    labels_one = tf.convert_to_tensor(np.ones(shape=(32,1)))

    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(labels_one,predictions_fake)
    return loss 

def discriminator_loss(predictions):
    # Create the label vector indicating which images are real and which are fake.
    labels_one = tf.convert_to_tensor(np.ones(shape=(32,1)))
    labels_zero = tf.convert_to_tensor(np.zeros(shape=(32,1)))
    labels = tf.concat([labels_zero, labels_one], axis=0)
    # Use binary cross entropy loss to compute the loss.
    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(labels,predictions)
    return loss


In [ ]:
!rm -rf ./logs/ 
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

%tensorboard --logdir logs/

tf.keras.backend.clear_session()

### Hyperparameters
num_epochs = 10
learning_rate = tf.optimizers.schedules.ExponentialDecay(0.001, 5000, 0.97, staircase=True)
running_average_factor = 0.95

# Initialize the Generator and Discriminator model.
generator = Generator()
discriminator = Discriminator()

# Define size of latent variable vector.
z_dim = 50
# Define random seed to use for generating 8 images for supervision.
seed = tf.random.normal(shape=[8,z_dim])

# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
gen_optimizer = tf.keras.optimizers.Adam(learning_rate)
dis_optimizer = tf.keras.optimizers.Adam(learning_rate)

step=0
for epochs in range(num_epochs):

  for real in train_ds:
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
            
            # Generate random noise vector. tf.random.normal()
            noise = tf.random.normal(shape=[32, z_dim])
            # Generate fake images with generator.
            fake = generator(noise, training=True)
            # Merge fake and real images to a long vector. tf.concat()
            images = tf.concat([fake, real], axis=0)
            # Compute output from discriminator.
            probs = discriminator(images, training=True)
            
            # Compute loss, compute gradients, apply gradients, store summaries.
            gen_loss = generator_loss(probs)
            dis_loss = discriminator_loss(probs)
            gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
            dis_gradients = dis_tape.gradient(dis_loss, discriminator.trainable_variables)

            gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
            dis_optimizer.apply_gradients(zip(dis_gradients, discriminator.trainable_variables))

            with train_summary_writer.as_default():
                tf.summary.scalar('generator_loss', gen_loss, step=step)
                tf.summary.scalar('discriminator_loss', dis_loss, step=step)
            
        # Every 100 steps generate images from the defined seed and 
        # store to supervise how well the generator works.
        if step % 100 == 0:
            fake = generator(seed, training=False)
            with test_summary_writer.as_default():
                tf.summary.image('fake_images', fake, step=step, max_outputs=8)
            
        step += 1

